In [5]:
from dotenv import load_dotenv
import os
import sys
load_dotenv()

global_path = os.getenv('global_path')
sys.path.append(global_path)

In [6]:
from athena import Strategy, Backtest
import pandas as pd

In [7]:
MULTIPLE_ASSETS_DATA = pd.DataFrame(
    index=['2023-10-18', '2023-10-19', '2023-10-20'],
    data={
        ('AAA','Open'): [100.0, 200.0, 300.0],
        ('AAA','High'): [100.0, 200.0, 300.0],
        ('AAA','Low'): [100.0, 200.0, 300.0],
        ('AAA','Close'): [100.0, 200.0, 300.0],
        ('AAA','Volume'): [2000000, 2500000, 3000000],
        ('BBB','Open'): [100.0, 200.0, 300.0],
        ('BBB','High'): [100.0, 200.0, 300.0],
        ('BBB','Low'): [100.0, 200.0, 300.0],
        ('BBB','Close'): [100.0, 200.0, 300.0],
        ('BBB','Volume'): [1500000, 1700000, 1600000]
    }
)

In [8]:
MULTIPLE_ASSETS_DATA

AAA                                  BBB                       \
             Open   High    Low  Close   Volume   Open   High    Low  Close   
2023-10-18  100.0  100.0  100.0  100.0  2000000  100.0  100.0  100.0  100.0   
2023-10-19  200.0  200.0  200.0  200.0  2500000  200.0  200.0  200.0  200.0   
2023-10-20  300.0  300.0  300.0  300.0  3000000  300.0  300.0  300.0  300.0   

                     
             Volume  
2023-10-18  1500000  
2023-10-19  1700000  
2023-10-20  1600000

In [5]:
# 这是对于多仓的测试
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        self.open(symbol='AAA', size = 1/10, price=record[('AAA','Open')], short=False, is_fractional=True)
        self.open(symbol='BBB', size = 1/10, price=record[('BBB','Open')], short=False, is_fractional=True)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0.1, cash=1000)
res = backtest.run()

所有未平仓持仓已经清算完毕！
回测结束，总资金: 1288.6756363636364


In [5]:
# 这是对于空仓的测试
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        
        self.open(symbol='AAA', size = 1/10, price=record[('AAA','Open')], short=True, is_fractional=True)
        self.open(symbol='BBB', size = 1/10, price=record[('BBB','Open')], short=True, is_fractional=True)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0.1, cash=1000)
res = backtest.run()

所有未平仓持仓已经清算完毕！
回测结束，总资金: 457.8574545454545


In [ ]:
# order_target_percent增仓
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        if i == 0:
            self.order_target_percent(symbol='AAA', target_percent=0.1, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        
        if i == 2:
            self.order_target_percent(symbol='AAA', target_percent=0.5, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0.1, cash=1000)
res = backtest.run()

买完后可用资金 900.0
买完后可用资金 540.9090909090909
所有未平仓持仓已经清算完毕！
回测结束，总资金: 1080.1652892561983


In [4]:
# order_target_percent减仓
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        
        print("可供使用用资金", self.broker.cash)
        print("持仓价值", self.broker.assets_value) # 问题是到了新的一天我的持仓价值还没有更新

        if i == 0:
            # 这里出现了个bug，target_value要下1单，但是1被当作分数下了全单
            self.order_target_percent(symbol='AAA', target_percent=0.5, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        
        if i == 2:
            self.order_target_percent(symbol='AAA', target_percent=0.1, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=1000)
res = backtest.run()

print("减仓仓位的记录: ", res.trades)

可供使用用资金 1000
持仓价值 0.0
买完后可用资金 500.0
可供使用用资金 500.0
持仓价值 500.0
可供使用用资金 500.0
持仓价值 1000.0
买完后可用资金 1350.0
所有未平仓持仓已经清算完毕！
回测结束，总资金: 2000.0
减仓仓位的记录:  [Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=100.0, close_price=300.0, position_size=2.8333333333333335, profit_loss=566.6666666666666, change_pct=2.0, trade_commission=0.0, cumulative_return=1566.6666666666665), Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=100.0, close_price=300.0, position_size=2.1666666666666665, profit_loss=433.3333333333333, change_pct=200.0, trade_commission=0.0, cumulative_return=1999.9999999999998)]
